# Overview
- This notebook fine-tunes the open-source models released by Google (gemma). The notebook can finetune both the 2b and 7b models.
- Finetuned on an A10G GPU
- Lora is used for finetuning.
- Dataset is Financial Phrase Bank from Kaggle.
- Inspired from (https://www.kaggle.com/code/lucamassaron/fine-tune-gemma-7b-it-for-sentiment-analysis)


In [1]:
from watermark import watermark
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # environment variable which tells pytorch to use the first GPU
os.environ["TOKENIZERS_PARALLELISM"] = "false" # Whether to parallelize tokenization process.

In [2]:
# To ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [3]:
# IMPORTS
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

import torch
import torch.nn as nn

import transformers
from transformers import (AutoModelForCausalLM, 
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments, 
                          pipeline, 
                          logging)

from datasets import Dataset
from peft import LoraConfig, PeftConfig
import bitsandbytes as bnb
from trl import SFTTrainer

from sklearn.metrics import (accuracy_score, 
                             classification_report, 
                             confusion_matrix)
from sklearn.model_selection import train_test_split
import accelerate

In [4]:
%load_ext watermark
%watermark -p torch,transformers,datasets,accelerate,trl,peft

torch       : 2.4.0
transformers: 4.43.2
datasets    : 2.20.0
accelerate  : 0.33.0
trl         : 0.9.6
peft        : 0.12.1.dev0



In [5]:
# Local file name.
filename = "all-data.csv"

# Read the file to pandas
df = pd.read_csv(filename, 
                 names=["sentiment", "text"],
                 encoding="utf-8", encoding_errors="replace")

# Create train, and test
X_train = list()
X_test = list()

# sample from each sentiment randomly and add 300 each to train and test.
for sentiment in ["positive", "neutral", "negative"]:
    train, test  = train_test_split(df[df.sentiment==sentiment], 
                                    train_size=300,
                                    test_size=300, 
                                    random_state=42)
    X_train.append(train)
    X_test.append(test)

X_train = pd.concat(X_train).sample(frac=1, random_state=10)
X_test = pd.concat(X_test)

# The ones that are not used will be eval set (randomly sample 50 from each sentiment)
eval_idx = [idx for idx in df.index if idx not in list(train.index) + list(test.index)]
X_eval = df[df.index.isin(eval_idx)]
X_eval = (X_eval
          .groupby('sentiment', group_keys=False)
          .apply(lambda x: x.sample(n=50, random_state=10, replace=True)))
X_train = X_train.reset_index(drop=True)


# Generate prompt for fine-tuning
def generate_prompt(data_point):
    return f"""
            Analyze the sentiment of the news headline enclosed in square brackets, 
            determine if it is positive, neutral, or negative, and return the answer as 
            the corresponding sentiment label "positive" or "neutral" or "negative"

            [{data_point["text"]}] = {data_point["sentiment"]}
            """.strip()

# Generate prompt for test
def generate_test_prompt(data_point):
    return f"""
            Analyze the sentiment of the news headline enclosed in square brackets, 
            determine if it is positive, neutral, or negative, and return the answer as 
            the corresponding sentiment label "positive" or "neutral" or "negative"

            [{data_point["text"]}] = 

            """.strip()

# Prompt from gemma model card
# def generate_prompt(data_point):
#     return f"""<start_of_turn>user
#             Analyze the sentiment of the news headline enclosed in square brackets, 
#             determine if it is positive, neutral, or negative, and return the answer as 
#             the corresponding sentiment label "positive" or "neutral" or "negative"
            
#             [{data_point["text"]}]<end_of_turn>
#             <start_of_turn>model
#             {data_point["sentiment"]}<end_of_turn>""".strip()

# def generate_test_prompt(data_point):
#     return f"""<start_of_turn>user
#             Analyze the sentiment of the news headline enclosed in square brackets, 
#             determine if it is positive, neutral, or negative, and return the answer as 
#             the corresponding sentiment label "positive" or "neutral" or "negative"
            
#             [{data_point["text"]}]<end_of_turn>
#             <start_of_turn>model
#             """.strip()

# Conver train and eval to Datasets
X_train = pd.DataFrame(X_train.apply(generate_prompt, axis=1), 
                       columns=["text"])
X_eval = pd.DataFrame(X_eval.apply(generate_prompt, axis=1), 
                      columns=["text"])

y_true = X_test.sentiment
X_test = pd.DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=["text"])

train_data = Dataset.from_pandas(X_train)
eval_data = Dataset.from_pandas(X_eval)

In [6]:
df.head(2)

,sentiment,text
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...


## Function to evaluate the performs of LLMs

In [7]:
def evaluate(y_true, y_pred):
    """
    Maps the labels to positive(2) , negative(0) or neutral(1).
    Calculate the overall accuracy.
    Calculate the overall accuracy of individual labels.
    Also generate classification report and confusion matrix.
    """
    labels = ['positive', 'neutral', 'negative']
    mapping = {'positive': 2, 'neutral': 1, 'none':1, 'negative': 0}
    
    def map_func(x):
        return mapping.get(x, 1)
    
    y_true = np.vectorize(map_func)(y_true)
    y_pred = np.vectorize(map_func)(y_pred)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Accuracy: {accuracy:.3f}')
    
    # Generate accuracy report
    unique_labels = set(y_true)  # Get unique labels
    
    for label in unique_labels:
        label_indices = [i for i in range(len(y_true)) 
                         if y_true[i] == label]
        label_y_true = [y_true[i] for i in label_indices]
        label_y_pred = [y_pred[i] for i in label_indices]
        accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {label}: {accuracy:.3f}')
        
    # Generate classification report
    class_report = classification_report(y_true=y_true, y_pred=y_pred)
    print('\nClassification Report:')
    print(class_report)
    
    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[0, 1, 2])
    print('\nConfusion Matrix:')
    print(conf_matrix)

# Load the model and tokensizer

In [13]:
# model_name = "/kaggle/input/gemma/transformers/7b-it/1"
model_name = "meta-llama/Meta-Llama-3.1-8B"

compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, # load in 4bit
    bnb_4bit_use_double_quant=False, # load in double quant
    bnb_4bit_quant_type="nf4", # nf4 type of quantization
    bnb_4bit_compute_dtype=compute_dtype, # use float16 for computation.
)

# Load the model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config, 
)

model.config.use_cache = False
model.config.pretraining_tp = 1

# Load the tokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          add_eos_token=True,
                                          trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.truncation_side = "left"

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

## Make predictions using the LLM

In [14]:
def predict(X_test, model, tokenizer):
    """
    Generate predictions. Input prompt is first converted to tokens,
    The tokens are sent to the LLM to generate an output.
    The output is then mapped to positive, negative, neutral
    """
    y_pred = []
    for i in tqdm(range(len(X_test))):
        prompt = X_test.iloc[i]["text"]
        input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
        outputs = model.generate(**input_ids, max_new_tokens=1, temperature=0.001)
        result = tokenizer.decode(outputs[0])
        answer = result.split("=")[-1].lower()
        if "positive" in answer:
            y_pred.append("positive")
        elif "negative" in answer:
            y_pred.append("negative")
        elif "neutral" in answer:
            y_pred.append("neutral")
        else:
            y_pred.append("none")
    return y_pred

In [15]:
y_pred = predict(X_test, model, tokenizer)
evaluate(y_true, y_pred)

  0%|          | 0/900 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  0%|          | 2/900 [00:00<01:17, 11.52it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  0%|          | 4/900 [00:00<01:17, 11.61it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  1%|          | 6/900 [00:00<01:18, 11.39it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  1%|          | 8/900 [00:00<01:17, 11.49it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  1%|          | 10/900 [00:00<01:17, 11.48

Accuracy: 0.399
Accuracy for label 0: 0.153
Accuracy for label 1: 0.050
Accuracy for label 2: 0.993

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.15      0.27       300
           1       0.43      0.05      0.09       300
           2       0.36      0.99      0.53       300

    accuracy                           0.40       900
   macro avg       0.59      0.40      0.30       900
weighted avg       0.59      0.40      0.30       900


Confusion Matrix:
[[ 46  19 235]
 [  0  15 285]
 [  1   1 298]]


### The accuracy across all the sentiments is 0.564. The accuracy is decent across all the labels. This is different than the instructionn tuned model.

# Finetuning the model
- Using the SFTTrainer.
- We will use the PEFT method, and set the parameters for LORA.
- Set the training parameters such as logging, number of epoch, batch_size, learning rate, etc.

In [16]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules="all-linear",
)

training_arguments = TrainingArguments(
    output_dir="logs",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="tensorboard",
    do_eval=False,
    evaluation_strategy="no",
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
    max_seq_length=1024,
)

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Detected kernel version 4.14.348, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [17]:
# Train model 
trainer.train()

# Save trained model
# trainer.model.save_pretrained("trained-model_it")

Step,Training Loss
25,1.615700
50,0.926000
75,0.870100
100,0.923100
125,0.818700
150,0.743400
175,0.698000
200,0.689800
225,0.698100
250,0.568700


TrainOutput(global_step=336, training_loss=0.7723437136127835, metrics={'train_runtime': 792.7185, 'train_samples_per_second': 3.406, 'train_steps_per_second': 0.424, 'total_flos': 1.0440591523356672e+16, 'train_loss': 0.7723437136127835, 'epoch': 2.986666666666667})

In [21]:
%load_ext tensorboard
%tensorboard --logdir logs/runs --bind_all

In [22]:
from tensorboard import notebook
print(notebook.list()) # View open TensorBoard instances
# !kill 16147

Known TensorBoard instances:
  - port 6006: logdir logs/runs (started 0:00:24 ago; pid 27343)
None


# Evaluate the model after training.

In [20]:
y_pred = predict(X_test, model, tokenizer)
evaluate(y_true, y_pred)


100%|██████████| 900/900 [01:45<00:00,  8.53it/s]

Accuracy: 0.859
Accuracy for label 0: 0.960
Accuracy for label 1: 0.827
Accuracy for label 2: 0.790

Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.96      0.95       300
           1       0.78      0.83      0.80       300
           2       0.85      0.79      0.82       300

    accuracy                           0.86       900
   macro avg       0.86      0.86      0.86       900
weighted avg       0.86      0.86      0.86       900


Confusion Matrix:
[[288  10   2]
 [ 13 248  39]
 [  4  59 237]]


### After training the performance improves significantly
- Overall accuracy is 0.847
- And the model performs well on all the labels.
- For some reason only when the packages were install from pip, could I get this to work. Also, peft had to be built from source (repo)